In [ ]:
#Importing Data from the CSV file
%matplotlib inline
import string
import math
import numpy as np

f_train = open("training.1600000.processed.noemoticon.csv", "r", encoding="ISO-8859-1")
X_train = f_train.readlines()
f_test = open("testdata.manual.2009.06.14.csv", "r")
X_test = f_test.readlines()

#Creating the List with just the tweets and finding the number of positive and negative classes. (TRAINING) 
#class_0 = number of classes with label = 0
#class_4 = number of classes with label = 4

class_0=class_4=0
tweets_train =[]
for x in X_train:
    a = x.split('","')
    if (a[0] == '"0'): class_0+=1
    else: class_4+=1
    tweets_train.append('%s"|"%s' % (a[0],a[-1]))

#Creating the List with just the tweets (TEST DATA)
tweets_test =[]
for x in X_test:
    a = x.split('","')
    tweets_test.append('%s"|"%s' % (a[0], a[-1]))
    
print("The length of the training set is = ", len(X_train))
print("The number of classes (label=4) =", class_4)
print("The number of classes (label=0) =", class_0)
print("The length of the test set is = ", len(X_test))

f_test.close()
f_train.close()

In [ ]:
#Writing the tweets to a file (not necessary - REMOVE if not used anywhere later)
!rm -rf tweet_test.txt
!rm -rf tweet_train.txt
f = open("tweet_train.txt", "w")
for x in tweets_train:
    f.write(x)
f.close()

f = open("tweet_test.txt", "w")
for x in tweets_test:
    f.write(x)
f.close()

In [ ]:
m= len(X_train)
phi_0 = class_0/m
phi_4 = class_4/m
v = len(dictionary)

In [ ]:
def train_nb_classifier(dictionary, dict_0, dict_4,len_tweets, class_0, class_4):
    theta_0 = {}
    theta_4 = {}
    n_0 = sum(len_tweets[0:class_0])
    n_4 = sum(len_tweets[class_0:len(len_tweets)])
    c=1
    for word in dictionary.keys():
            if word in dict_0:
                theta_0[word] = ((dict_0[word]+c)/(n_0 + v*c))
            else:
                theta_0[word] = ((c) / (n_0 + v*c))
            if word in dict_4:
                theta_4[word] = ((dict_4[word]+c)/(n_4 + v*c))
            else:
                theta_4[word] = ((c)/(n_4 + v*c))
    return theta_0,theta_4;

In [ ]:
theta_0, theta_4 = train_nb_classifier(dictionary, dict_0, dict_4, len_tweets, class_0, class_4)

In [ ]:
def test_data(tweets):
    pred_class=[]
    actual_class=[]
    for x in tweets:
        test_class0=test_class4=0
        x = x.rstrip("\n\r")
        x = x.split("|")
        twt = x[1].translate(str.maketrans('','',string.punctuation)).lower()
        words = twt.split() # if I use split(" "), it takes more than one white space as a word
        
        #Store actual class number in Test Data for forming Confusion matrix
        actual_class.append(x[0])
        
        #Finding probability of tweet being in a class         
        for w in words:
            if w in theta_0: test_class0 += math.log(theta_0[w])
            else: test_class0 += math.log(1)
            if w in theta_4: test_class4 += math.log(theta_4[w])
            else: test_class4 += math.log(1)
        test_class0 += math.log(phi_0)
        test_class4 += math.log(phi_4)
        #Classifying the probability into classes
        if (test_class0 > test_class4): pred_class.append('"0"')
        else: pred_class.append('"4"')

            
    actual_class=[0 if x=='"0"' else 1 for x in actual_class]
    pred_class=[0 if x=='"0"' else 1 for x in pred_class]
    test_error = sum(np.bitwise_xor(actual_class, pred_class))
    accuracy = ((len(tweets) - test_error)/len(tweets))*100
    return accuracy, actual_class, pred_class;

In [ ]:
train_accuracy,actual_class_train,pred_class_train = test_data(tweets_train)
print("Result (a) : The train accuracy of the model on raw data is = {}%".format(train_accuracy))

In [ ]:
test_accuracy, actual_class,pred_class = test_data(tweets_test)
print("Result (a) : The test accuracy of the model on raw data is = {}%".format(test_accuracy))

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
#nltk.download()

In [ ]:
tweets_train_pd = pd.Series(tweets_train)

In [ ]:
train_classes = tweets_train_pd.apply(lambda x: (x.split("|")[0]))

In [ ]:
print(train_classes)

In [ ]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
tweets_train_pd['tokenize_data'] = tweets_train_pd.apply(lambda x: '\n'.join([' '.join(token) for token in ([tknzr.tokenize(x.split("|")[1])])]))


In [ ]:
tweets_train_pd['count'].mask(train_classes == '"4"')

In [ ]:
tweets_train_pd['tokenize_data']=tweets_train_pd['tokenize_data'].str.rstrip("\n\r")
tweets_train_pd['clean_data'] = tweets_train_pd['tokenize_data'].str.translate(str.maketrans('','',string.punctuation)).str.lower().str.split()
#Use lambda function to apply the same expression on all the strings together. 
#function takes one string at a time and returns a string wihtout stop words
stop_words=set(stopwords.words('english'))
tweets_train_pd['no_stop_words'] = tweets_train_pd['clean_data'].apply(lambda x: [word for word in x if word not in stop_words and word.isalpha()])
wordnet_lemmatizer = WordNetLemmatizer()
tweets_train_pd['lemma1'] = tweets_train_pd['no_stop_words'].apply(lambda x: [wordnet_lemmatizer.lemmatize(word, pos ="n") for word in x])
tweets_train_pd['lemma2'] = tweets_train_pd['lemma1'].apply(lambda x: [wordnet_lemmatizer.lemmatize(word, pos ="v") for word in x])
tweets_train_pd['lemma'] = tweets_train_pd['lemma2'].apply(lambda x: [wordnet_lemmatizer.lemmatize(word, pos ="a") for word in x])

In [ ]:
from collections import Counter
tweets_train_pd['count'] = tweets_train_pd['lemma'].apply(lambda x: Counter(x)) 
dictionary=tweets_train_pd['count'].sum()
dictionary = dict(dictionary)


In [ ]:
tweets_train_pd['count'][::10].sum()